<h1>Segmenting and Clustering Neighborhoods in Toronto</h1>
<h4>Probelm 1 : Explore and cluster the neighborhoods in Toronto </h4>

In [1]:
#import packages
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

1. Start by creating a new Notebook for this assignment.
2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(url).text

Use BeautifulSoup package to pasre html tags in wiki webpage

In [3]:
soup = bs(source, 'html')
table=soup.find('table')

Create dataframe with three columns: PostalCode, Borough, and Neighborhood

In [4]:
#create dataframe with three columns: PostalCode, Borough, and Neighborhood
column_names = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)

Fill data it into dataframe

In [5]:
#Search data and fill it into dataframe
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [6]:
#get data frame shape
df.shape

(180, 3)

3. To create the above dataframe:
<ol>
    <li>The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood</li>
    <li>Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.</li>
    <li>More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.</li>
<li>If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.</li>
<li>Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.</li>
<li>In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.</li>
 </ol>

Remove the record where value of Borough is Not Assigned

In [7]:
#remove Borough that is Not assigned
df = df[df['Borough']!='Not assigned']
print("Shape : ", df.shape)
df = df.reset_index(drop=True)
df.head()

Shape :  (103, 3)


,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Assigne the Neighborhood equals Borough for records where value of Neighborhood is Not Assigned

In [8]:
#remove Neighborhood that is Not assigned
flg = df[df['Neighborhood']=='Not assigned']
df[flg]['Neighborhood'] = df[flg]['Borough']
df.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Remove all duplicate records

In [9]:
#remove duplicate records
df1 = df.drop_duplicates()
df1 = df1.reset_index(drop=True)
df1.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Find if multiple record of Postalcode exists, if exists then applend the Neghborhood column  

In [10]:
temp = df1.groupby('Postalcode').count()
temp['Borough'].max()

1

No multiple recods found. Hence, no need to do any thing. Data is clear.

In [11]:
df1.shape

(103, 3)

<h4>Problem 2 : Get the latitude and the longitude coordinates of each neighborhood</h4>

import geocoder # import geocoder

def get_geocode(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

get_geocode('M5G')

Get coordinates of each neighborhood from csv file

In [12]:
cord_df=pd.read_csv('http://cocl.us/Geospatial_data')
cord_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Combine both dataframe i.e. postal code and coordinate

In [14]:
cord_df.rename(columns={'Postal Code':'Postalcode'},inplace=True)
cord_df_final = pd.merge(cord_df, df1, on='Postalcode')
cord_df_final.head()

,Postalcode,Latitude,Longitude,Borough,Neighborhood
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge"
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae


Rearrange columns

In [16]:
cord_df_final = cord_df_final[['Postalcode','Borough','Neighborhood','Latitude','Longitude']]
cord_df_final.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Check shape

In [17]:
cord_df_final.shape

(103, 5)